# RiskRadar — Social-Only Narrative Risk Triage

**Setup:** Upload your data files, install deps, run pipeline, launch UI.

**Required files** (place in `data/` folder):
- `posts.jsonl` — social media posts
- `entities_seed.csv` — entity catalog
- `authors.csv` — author profiles

Run cells top-to-bottom. Cell 3 is optional (re-runs pipeline). Cell 5 launches the Streamlit UI.

In [ ]:
# Cell 1: Install dependencies
!pip install -q rapidfuzz scikit-learn streamlit python-dotenv anthropic pyngrok

In [ ]:
# Cell 2: Configuration
import os

# Leave empty for fuzzy-only mode (free, no API cost)
os.environ["ANTHROPIC_API_KEY"] = ""  # or paste "sk-ant-..."

# For Streamlit tunnel (get free token at https://dashboard.ngrok.com)
NGROK_TOKEN = ""  # paste your ngrok auth token

In [ ]:
# Cell 3: Run pipeline (optional — pre-computed outputs ship with repo)
# Uncomment to re-generate outputs from scratch.

# !python run_pipeline.py

In [ ]:
# Cell 4: Quick check — verify outputs
import json
from pathlib import Path

scored_dir = Path("outputs/scored")
if scored_dir.exists():
    files = sorted(scored_dir.glob("*_scored.json"))
    print(f"Scored entities: {len(files)}")
    for f in files[:5]:
        data = json.loads(f.read_text())
        entity = f.stem.replace('_scored', '')
        top = data[0] if data else {}
        print(f"  {entity}: {len(data)} narratives | top risk: {top.get('risk_score', '?')}")
    if len(files) > 5:
        print(f"  ... and {len(files) - 5} more")
else:
    print("No outputs found. Uncomment and run Cell 3 first.")

In [ ]:
# Cell 5: Launch Streamlit UI
from pyngrok import ngrok
import time

ngrok.kill()
time.sleep(2)

if NGROK_TOKEN:
    ngrok.set_auth_token(NGROK_TOKEN)
    !nohup streamlit run app.py --server.port 8501 --server.headless true &
    time.sleep(5)
    public_url = ngrok.connect(8501)
    print(f"\n Open: {public_url}")
    print("Keep this Colab tab open — closing it kills the tunnel.")
else:
    print("Set NGROK_TOKEN in Cell 2 to launch the UI.")
    print("Get a free token: https://dashboard.ngrok.com/get-started/your-authtoken")